Importing necessary libraries:

In [ ]:
# data manipulation libraries
import pandas as pd
import numpy as np
import xarray as xr
import geopandas as gpd

from shapely.geometry import Point


# built-in libraries
from itertools import product
import sys
import os
import subprocess
import sqlite3
import datetime

# import HydroErr
import HydroErr as he

Here, we calibrate the models based on the `KGE` metric.

In [ ]:
# define calibration model
# basic instructions
basin_name = 'calgary'
model = 'mesh'
gauges = ['05BH004']

# start and end date for the calibration period
cal_start = '1985-01-01T00:00:00'
cal_end = '2014-12-31T23:00:00'

In [ ]:
# paths
home = os.getenv('HOME')
obs_path = f'{home}/downloads/Hydat.sqlite3'

# read observed data
conn = sqlite3.connect(obs_path) # locate your HYDAT sqlite database

In [ ]:
def get_the_daily_dataframe(
    df: pd.DataFrame,
    regex_str: str,
    col: str,
    *args,
    **kwargs,
) -> pd.DataFrame:
    
    # filter and trim columns
    df = df.filter(regex=regex_str, axis=1) # extract the columns
    df.columns = df.columns.str.replace(r'\D', '', regex=True) # remove non-digits
    df = df.stack(future_stack=True) # stack without dropping
    df.index.names = ['STATION_NUMBER', 'YEAR', 'MONTH', 'DAY'] # assign index names
    df = df.reset_index() # reset index to add another level
    df['DATE'] = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']].astype(str).agg('-'.join, axis=1), errors='coerce') # define date column
    df.drop(columns=['YEAR', 'MONTH', 'DAY'], inplace=True) # drop unnecessary columns
    df.dropna(subset=['DATE'], inplace=True) # remove invalid dates
    df.set_index(keys=['STATION_NUMBER', 'DATE'], drop=True, inplace=True) # set index levels
    df.columns = [col] # assing column name
    
    # pivot data to look nice
    df = df.unstack(level='STATION_NUMBER')
    df = df.reorder_levels(order=[1,0], axis=1)
    
    return df


def extract_station_daily_flow(
    station: str,
    connection: sqlite3.Connection,
    start_date: str = '1850-01-01',
    end_date: str = str(datetime.datetime.now().date()),
    *args,
    **kwargs,
) -> pd.DataFrame:
    
    '''
    This function simply extracts data from the HYDAT sqlite3 database
    '''
    
    # read station data
    df = pd.read_sql_query(f"SELECT * FROM DLY_FLOWS WHERE STATION_NUMBER LIKE '%{station}%'", connection)
    
    # set index
    df.set_index(keys=['STATION_NUMBER', 'YEAR', 'MONTH'], drop=True, inplace=True)
    
    # get the FLOW and FLAG
    df_flow = get_the_daily_dataframe(df, r'^FLOW\d', 'FLOW')
    df_flag = get_the_daily_dataframe(df, r'^FLOW_.', 'FLAG')
    
    df = pd.concat([df_flow, df_flag], 
                   axis=1)
    df.sort_index(axis=0, inplace=True)
    df.sort_index(axis=1, level=0, ascending=False, inplace=True)
    
    df = df.loc[start_date:end_date, :]
    
    return df

def extract_station_coords(
    station: str,
    connection: sqlite3.Connection,
) -> dict:
    '''
    returns `latitude` and `longitude` values for `station`
    '''
    
    # read the specs
    df = pd.read_sql_query(f"SELECT * FROM STATIONS WHERE STATION_NUMBER LIKE '%{station}%'", conn)
    
    # rename the columns to their lowercase equivalent
    df.rename(
        columns={
            'LATITUDE': 'latitude',
            'LONGITUDE': 'longitude',
        },
        inplace=True,
    )
    
    # making a dictionary out of the values
    vals_dict = df.loc[:, ['latitude', 'longitude']].to_dict(orient='list')
    
    # return the values in form of a dictionary
    vals_dict = {k:v[0] for k,v in vals_dict.items()}
    
    return vals_dict

In [ ]:
# observation time-series
# obs = extract_station_daily_flow(gauges[0], conn, start_date=cal_start, end_date=cal_end)
# easier solution
obs = pd.read_csv(
    './etc/obs.csv',
    index_col=[0],
    header=[0,1],
    parse_dates=True,
)

# observations gauge locations
# obs_location = extract_station_coords(gauges[0], conn)
# easier solution
obs_location = {'latitude': 51.05, 'longitude': -114.05}

In [ ]:
if model == 'mesh':
    # run MESH
    # the module related functions need modifications on individual clusters
    command = (
    'ml restore scimods; '
    f'cd ./{model}; '
    'mpirun -n 50 ./mpi_sa_mesh;'
    )
    result = subprocess.run(
        command,
        capture_output=True,
        shell=True
    )
    # specify the output file
    sim_path = f'./{model}/results/QO_PTS-1440M_AVG.csv'

    # read the simulated results
    try:
        sim = pd.read_csv(
            sim_path,
            header=None,
            index_col=0,
            parse_dates=True,
            date_format='%Y/%m/%d %H:%M:%S.000'
        )
    except: # if model crashes, so sim_path becomes corrupt
        with open(f"./{model}/results/kge_2012.csv", "w") as file:
            # write a terrible metric value 
            # to indicate model crash
            # OSTRICH assumes a minimization problem
            file.write("2000\n")
            sys.exit()

    # drainage database path
    ddb_path = f'./{model}/MESH_drainage_database.nc'
    ddb = xr.open_dataset(
        ddb_path,
    )
    
    # subbasin geometries path
    geoms = gpd.read_file(
        f'./{model}/geometries/bcalgary_subbasins.shp'
    )
    
elif model == 'summa':
    pass
elif model == 'hype':
    pass

In [ ]:
# finding intersection of the gauge location with the subbasins
# defining gauge location Point object
point = Point(obs_location['longitude'], obs_location['latitude'])

# intersecting with the subbasin polygons
intersecting_polygons = geoms[geoms.geometry.intersects(point)]

# extracting the sub-basin ID (COMID) in this case
basin_id = intersecting_polygons.COMID.values[0]

# extracting sub-basin rank
subbasin_rank = int(ddb['Rank'].sel(subbasin=basin_id).values)

In [ ]:
# prepareing the `sim' Series
sim_df = sim.loc[:, subbasin_rank]
sim_df.index.name = 'time'
sim_df.name = 'sim'
sim_df = sim_df.resample('D').mean()

# preparing the `obs' Series
idx = pd.IndexSlice
obs_df = obs.loc[:, idx[gauges[0], 'FLOW']]
obs_df.index.name = 'time'
obs_df.name = 'obs'

In [ ]:
# calculating metrics of interest
for g in gauges:
    # define dataframes
    sim_df = sim_df.loc[cal_start:cal_end]
    obs_df = obs_df.loc[cal_start:cal_end]
    
    # calculate metrics - minimization problem
    try:
        kge = -1 * (he.kge_2012(sim_df, obs_df))
    except:
        with open(f"./{model}/results/kge_2012.csv", "w") as file:
            # write a terrible metric value 
            # to indicate model crash
            # OSTRICH assumes a minimization problem
            file.write("2000\n")
            sys.exit()

# printing to a file
with open(f"./{model}/results/kge_2012.csv", "w") as file:
    file.write(f"{kge}\n")  # Using an f-string to format the float